# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1['128']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201002_073628', '20201029_180105_no_adv_loss', '20…

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201121_050116_full_loss_b64_lr0.0002_using-var


In [12]:
# ### Read loss data
# with open (result_dir+'/metrics.pickle','rb') as f:
#     metrics_dict=pickle.load(f)
    
# df_metrics=pd.DataFrame([])
# for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
# df_metrics['step']=np.arange(df_metrics.shape[0])
# # df_metrics=df_metrics.dropna()

## Plot Losses

In [18]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [19]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
124991,124991.0,39.0,0.147305,0.043159,0.190464,6.377608,89.524605,83.146996,-2.951660,83.481117,-2.237294,8.257886,-6.174417,-6.334901,0.124559
124992,124992.0,39.0,0.224993,0.184241,0.409234,9.955685,92.017014,82.061325,-2.962397,81.773262,-2.666686,8.473328,-4.952648,-9.954659,0.125378
124993,124993.0,39.0,0.204236,0.122393,0.326629,8.808847,92.775185,83.966339,-2.651654,77.398026,-2.803541,5.877655,-8.793900,-8.804411,0.125109
124994,124994.0,39.0,0.294146,0.141892,0.436038,3.458988,86.547020,83.088036,-2.763365,85.817055,-2.166744,5.283353,-7.192931,-3.201501,0.125378
124995,124995.0,39.0,0.201365,0.468589,0.669954,14.970137,95.626915,80.656776,-2.756321,84.133179,-2.596920,10.169331,-2.320587,-14.970133,0.125664
124996,124996.0,39.0,1.519649,0.213968,1.733617,0.297599,83.743538,83.445938,-2.728753,84.533241,-2.429098,-0.066796,-14.059924,4.126239,0.124798
124997,124997.0,39.0,0.247387,5.525835,5.773222,42.051003,124.962593,82.911591,-2.645559,86.833298,-2.177287,21.735661,5.522330,-42.051003,0.123669
124998,124998.0,39.0,12.938555,0.616187,13.554742,17.280067,101.515450,84.235382,-2.754203,82.321030,-1.330554,-13.093921,-39.701752,-17.279985,0.124558
124999,124999.0,39.0,0.239105,0.242905,0.482009,3.879077,86.395317,82.516243,-2.926435,82.350967,-2.661517,16.595394,-15.339330,-3.057191,0.123839
125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.759354,-1.879380,NaN,NaN,NaN,NaN


In [15]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [20]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-2.976103136539459


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
5296,5296.0,1.0,0.473061,0.163848,0.636909,34.649372,118.488342,83.838966,-3.335656,86.395393,-3.484381,23.404121,-38.424324,-34.278431,0.123672
5511,5511.0,1.0,3.525789,0.695135,4.220923,26.720661,111.078659,84.358002,-3.132360,86.233627,-3.418319,5.874992,-42.939037,-26.581205,0.126024
5361,5361.0,1.0,0.875641,0.558731,1.434372,21.605639,111.895851,90.290215,-3.056941,100.863098,-3.415606,29.555634,-25.812050,-21.499516,0.123642
5459,5459.0,1.0,1.847279,0.979584,2.826863,44.708679,140.075165,95.366486,-2.606776,91.607193,-3.363229,12.427042,-59.521706,-44.708679,0.126197
5607,5607.0,1.0,0.306734,1.184904,1.491638,22.596859,105.972275,83.375420,-3.213954,85.159416,-3.310261,45.347454,-34.027149,-22.230953,0.122939
95713,95713.0,30.0,0.439693,0.148709,0.588402,6.412812,92.127380,85.714569,-2.835889,83.579277,-3.305142,2.883978,-12.987633,-6.299338,0.129152
5643,5643.0,1.0,0.731660,1.803010,2.534670,30.156301,144.602310,114.446014,-2.617860,111.485535,-3.297609,32.054699,-13.464179,-30.101704,0.123453
5356,5356.0,1.0,0.842997,2.235330,3.078327,13.239735,98.629425,85.389687,-3.017623,85.659477,-3.282836,47.161079,-18.539833,-12.352215,0.124713
105582,105582.0,33.0,0.205473,0.295959,0.501432,12.724692,96.433067,83.708374,-2.483064,82.465584,-3.277526,11.428355,-10.805768,-12.724035,0.129125
116536,116536.0,37.0,0.346909,0.167258,0.514167,6.027350,89.419243,83.391891,-3.046691,86.317566,-3.256988,6.110058,-8.292194,-5.956536,0.131999


In [21]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
5296,5296.0,1.0,0.473061,0.163848,0.636909,34.649372,118.488342,83.838966,-3.335656,86.395393,-3.484381,23.404121,-38.424324,-34.278431,0.123672
5511,5511.0,1.0,3.525789,0.695135,4.220923,26.720661,111.078659,84.358002,-3.132360,86.233627,-3.418319,5.874992,-42.939037,-26.581205,0.126024
5361,5361.0,1.0,0.875641,0.558731,1.434372,21.605639,111.895851,90.290215,-3.056941,100.863098,-3.415606,29.555634,-25.812050,-21.499516,0.123642
5459,5459.0,1.0,1.847279,0.979584,2.826863,44.708679,140.075165,95.366486,-2.606776,91.607193,-3.363229,12.427042,-59.521706,-44.708679,0.126197
5607,5607.0,1.0,0.306734,1.184904,1.491638,22.596859,105.972275,83.375420,-3.213954,85.159416,-3.310261,45.347454,-34.027149,-22.230953,0.122939
95713,95713.0,30.0,0.439693,0.148709,0.588402,6.412812,92.127380,85.714569,-2.835889,83.579277,-3.305142,2.883978,-12.987633,-6.299338,0.129152
5643,5643.0,1.0,0.731660,1.803010,2.534670,30.156301,144.602310,114.446014,-2.617860,111.485535,-3.297609,32.054699,-13.464179,-30.101704,0.123453
5356,5356.0,1.0,0.842997,2.235330,3.078327,13.239735,98.629425,85.389687,-3.017623,85.659477,-3.282836,47.161079,-18.539833,-12.352215,0.124713


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
99580,99580.0,31.0,0.652009,0.295070,0.947079,11.083920,94.291016,83.207100,-2.208390,74.407730,-2.172292,3.964501,-11.243237,-11.024734,0.129280
120248,120248.0,38.0,0.058098,0.374799,0.432897,13.299954,98.385544,85.085587,-2.694031,75.106041,-2.663162,9.740522,-3.949594,-13.299695,0.127934
114424,114424.0,36.0,0.251797,0.236226,0.488023,12.300739,91.711227,79.410484,-2.643130,75.127823,-2.230327,6.656033,-10.223900,-12.285076,0.130392
69492,69492.0,22.0,0.655806,0.331027,0.986833,16.489418,95.792320,79.302902,-2.080410,75.213257,-2.068571,4.407508,-21.838367,-16.343090,0.126132
72757,72757.0,23.0,0.162489,0.241126,0.403615,9.747093,90.159004,80.411911,-0.892705,75.406158,-0.864648,10.887531,-9.030655,-9.739256,0.131950
121269,121269.0,38.0,0.165686,0.090762,0.256448,9.881229,88.242966,78.361732,-2.548288,75.604004,-2.662696,9.315769,-9.038465,-9.878053,0.124577
81482,81482.0,26.0,0.251573,0.352325,0.603898,10.021318,94.599281,84.577965,-1.466440,75.640465,-1.648792,8.378956,-9.355268,-10.010149,0.130718
107515,107515.0,34.0,0.108523,0.143649,0.252172,9.456032,92.495308,83.039276,-2.707852,75.679085,-2.376166,9.275326,-9.254330,-9.448670,0.130353


In [22]:
df_metrics[df_metrics.step.isin([44861,44862,44863])]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
44861,44861.0,14.0,0.241472,0.537670,0.779142,21.293617,113.992645,92.699028,-1.308178,90.684219,-1.188350,14.819358,-19.398743,-21.292692,0.127613
44862,44862.0,14.0,0.140692,0.186705,0.327397,19.130674,104.266838,85.136162,-1.233545,85.977425,-1.214888,11.196203,-19.523083,-19.130573,0.125900
44863,44863.0,14.0,0.257212,0.308470,0.565682,14.521505,99.459198,84.937691,-1.405668,85.687386,-1.140568,11.341042,-17.093212,-14.516122,0.123502


## Read stored chi-squares for images

In [61]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting


col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


39

In [58]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [54]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [55]:
df_sliced=w.result
df_sliced

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1963,31-98150,0.009207,0.767348,28.843225,0.004708,0.002128,0.002371,0.003120,9.169238,83.814067,31,98150
1640,26-82050,0.009852,0.253690,26.809908,0.002339,0.004979,0.002534,0.001986,8.615924,25.609580,26,82050
1798,28-89950,0.011678,0.510477,29.099435,0.005223,0.003445,0.003011,0.004405,9.595539,25.489336,28,89950
2267,36-113300,0.012582,0.756124,30.845821,0.002839,0.004811,0.004932,0.000285,9.800851,58.047587,36,113300
2166,34-108250,0.012633,0.480830,27.951553,0.002172,0.005290,0.005171,0.000562,10.411800,53.257019,34,108250
2354,37-117650,0.012789,0.405356,29.527308,0.003608,0.006389,0.002792,0.002601,8.711811,29.370501,37,117650
1751,28-87600,0.013425,0.817202,29.835788,0.007466,0.002433,0.003526,0.005565,12.113312,18.899760,28,87600
2299,36-114900,0.014200,0.340777,28.817668,0.003642,0.003966,0.006592,0.002594,13.573229,11.760973,36,114900
2072,33-103550,0.015295,0.688210,32.346116,0.011491,0.001704,0.002100,0.006160,20.164774,14.095489,33,103550
1910,30-95550,0.017361,0.745807,24.910623,0.005843,0.002272,0.009247,0.004605,12.970490,28.422039,30,95550


In [57]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, 39]
(2502, 21)
{'chi_1': 0.11547852029995515, 'chi_spec1': 1.5805721665689387}
[0, 39]
(2502, 21)
{'chi_1': 0.34326309781771336, 'chi_spec1': 4.139088960862047}
[0, 39]
(2502, 21)
{'chi_1': 0.11547852029995515, 'chi_spec1': 1.5805721665689387}
[ 42250  82050  98150 112600 112850 117600]


In [23]:
# best_step=[9386]
# best_step=np.arange(40130,40135).astype(int)

In [59]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 21)
[(13, 42250), (26, 82050), (31, 98150), (36, 112600), (36, 112850), (37, 117600)]


In [62]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
844,13-42250,0.093650,3.508616,31.413529,0.067056,0.025580,0.001014,0.019686,12.651756,70.948387,13,42250
1640,26-82050,0.009852,0.253690,26.809908,0.002339,0.004979,0.002534,0.001986,8.615924,25.609580,26,82050
1963,31-98150,0.009207,0.767348,28.843225,0.004708,0.002128,0.002371,0.003120,9.169238,83.814067,31,98150
2253,36-112600,0.009761,1.365683,28.691998,0.005566,0.001823,0.002371,0.004782,9.627519,85.074683,36,112600
2258,36-112850,0.126352,1.731625,31.427766,0.113083,0.012334,0.000935,0.044201,15.347205,11.608594,36,112850
2353,37-117600,0.040321,0.242511,28.327872,0.022407,0.006968,0.010946,0.021653,11.994908,16.629723,37,117600


### Plots 2: stored data

In [63]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[-4000:][:,0,:,:]

print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=post.f_compute_hist_spect(s_val,bins)
del(s_val)

(4000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [64]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec'],"Invalid mode %s"%(plot_type)

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    
#     plt.xscale('symlog',linthreshx=30)

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')

In [65]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec']))

interactive(children=(SelectMultiple(description='labels_list', options=('13-42250', '26-82050', '31-98150', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [29]:
# step=best_step[-1]
step=117600
fname='*gen_img_epoch-*_step-{0}.npy'.format(step)
# fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(result_dir+'/images/'+fname)[0]
print(fle)
imgs=np.load(fle)
print(imgs.shape)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201121_050116_full_loss_b64_lr0.0002_using-var/images/gen_img_epoch-37_step-117600.npy
(64, 128, 128)


In [30]:
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
df_metrics[df_metrics.step<2000]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
0,0.0,0.0,155.384521,8994.041016,9149.425781,0.000000,86.405716,86.405716,2.091570,NaN,NaN,10250.341797,9178.380859,8575.310547,16.994106
1,1.0,0.0,196.520554,9452.510742,9649.031250,0.000000,inf,inf,1.997464,inf,2.475530,10238.923828,9532.652344,9144.101562,0.465852
2,2.0,0.0,145.714554,9150.929688,9296.644531,0.000000,112.715851,112.715851,2.111522,inf,2.132113,10227.984375,9256.610352,8818.741211,0.399666
3,3.0,0.0,156.997803,8639.974609,8796.972656,0.000000,92.414574,92.414574,2.142157,104.134933,2.044554,10240.458984,8736.583008,8383.701172,0.405215
4,4.0,0.0,150.957550,8811.884766,8962.842773,0.000000,87.315872,87.315872,2.100545,87.325592,2.294785,10213.558594,8902.645508,8509.358398,0.397767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995.0,0.0,2.332118,22.379145,24.711262,88.450546,172.453705,84.003159,1.426435,84.166122,1.458471,73.462387,-8.319846,-88.012779,0.403768
1996,1996.0,0.0,7.975113,3.122581,11.097694,97.729805,181.735413,84.005600,1.542325,86.985519,1.418108,32.516716,-54.417641,-97.729805,0.415173
1997,1997.0,0.0,17.508060,6.264686,23.772747,48.942490,135.066467,86.123985,1.571042,85.655090,1.506533,-1.515057,-52.006775,-44.579414,0.417067
1998,1998.0,0.0,9.641037,7.489760,17.130798,68.521042,154.226471,85.705429,1.491595,92.041786,1.432904,26.949451,-83.883057,-63.937595,0.449832
